__v1.3__
<br/>
Due to issues found in the POI aggregation pipeline (regarding low data coverage for non-programme countries) this notebook has been updated to reflect that POIs are now all stored in one file.
Also to mititage some of the issues with bad user data the stored geomeasures now include a coverage column indicating for how many hours of the day we have data coverage for.

__v1.2__
<br/>
Changing the format of save-files, such that we now are not storing urban (1)/rural (0) indicator, home location (lat,lon) for each day for each user, so we later can easier merge with other data. Also streamlined the calculation piece, such that poverty look-up is also done here

__v1.1__
<br/>
Put everyting into functions to automate code

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import shapely
import operator
import rasterio
import math
import copy
from collections import Counter, defaultdict

### arguments + functions

In [9]:
def load_interval_data(path_,day_,file_):
    # load interval data
    interval_data = pd.read_csv(path_ + day_ + "/" + file_)

    return interval_data

def load_stops_data(path_,day_,file_):
    # load data for centroids
    stops_data = pd.read_csv(path_ + day_ + "/" + file_)    
    
    return  stops_data

def haversine_distance(lat1,lon1,lat2,lon2):
    radius = 6371 # radius of earth in km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

def infer_home_label(types_,labels_,admins_,starts_,ends_):
    home = Counter()
    for i,l in enumerate(labels_):
        if l != -1:
            home[(l,admins_[i])] += ends_[i] - starts_[i]
    
    if home != Counter():
        return home.most_common(1)[0]
    else:
        return ((-1,-1),-1)
    
def calculate_radius_of_gyration(labels_,positions_,home_label):
    # create dict of positions
    loc_dict = dict(zip(labels_,positions_))

    # only calculate radius of unique set of points
    labels_ = set(labels_)
    # remove the home location
    labels_.remove(home_label)
    # count number of labels
    n = len(labels_)
    if n > 0:
        # calculate distance between labels and home label
        rg_ = []
        home_lat, home_lon = loc_dict[home_label]
        for l in labels_:
            if l != -1:
                lat_,lon_ = loc_dict[l]
                # add to list
                rg_.append(haversine_distance(home_lat,home_lon,lat_,lon_)**2)
        # calculate radius using (eq S2) from Gonzales et al. Understanding individual human mobility patterns
        return np.sqrt(1/float(n)*sum(rg_))
    else:
        return -1  # person has not moved
    
def get_value_from_raster(raster,band,lat,lon,max_row,max_col):
    row,col = raster.index(lon,lat)

    if  0 <= row < max_row and 0 <= col < max_col:
        return band[row, col]
    else:
        return -1
    
def load_urban_raster():
    # open raster image containing urban/rural info
    raster = rasterio.open('/home/vsekara/mb_data/magicbox-public/settlements/GHS_SMOD_1km/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0_RPJ.tif')

    # read settlement raster
    band = raster.read(1)
    # modify array
    band[band <= 10] = -1 # nothing
    band[(11 <= band) & (band <= 13)] = 0 # rural
    band[21 <= band] = 1 # urban
    max_row, max_col = band.shape
    
    return raster, band, max_row, max_col
    
def load_poverty_raster(country):
    # load poverty data
    pov_data = {
        'NGA':'/home/vsekara/mb_data/magicbox-public/poverty/NGA/worldpop/nga10povcons200.tif',
        'IDN':'/home/vsekara/mb_data/magicbox-public/poverty/IDN/idn_poverty_rate.tif',
        'MOZ':'/home/vsekara/mb_data/magicbox-public/poverty/MOZ/AtlasAI/mozambique_rpj.tif',
        'CIV':'/home/vsekara/mb_data/magicbox-public/poverty/CIV/AtlasAI/cotedivoire_rpj.tif',
        'COL':'/home/vsekara/mb_data/magicbox-public/poverty/COL/DANE/poverty_colombia_mun.tif',
    }

    raster_path = pov_data[country]
    
    # open raster image
    raster = rasterio.open(raster_path)
    # read poverty raster
    band = raster.read(1)
    # threshold array
    band[band < 1e-3] = 0
    max_row, max_col = band.shape
    
    return raster, band, max_row, max_col

#countries_with_povdata = {'NGA','IDN','COL','CIV','MOZ'}
countries_with_povdata = {'NGA','COL','CIV','MOZ'}

## calculate 1) dist travlled 2) time spent at home 3) r_gyration, 4) no. pois

In [10]:
def calculate_geostats(country,t_day,t_end,admin='admin1'):

    # define paths
    if admin == "admin1":
        stop_path = "/home/vsekara/mb_data/cuebiq/nCoV/POI/%s/stops_gz/" % country
        admin_key = 'GID_1'
    elif admin == 'admin2':
        stop_path = "/home/vsekara/mb_data/cuebiq/nCoV/POI/%s/stops_gz_level_2/" % country
        admin_key = 'GID_2'
    interval_path = "/home/vsekara/mb_data/cuebiq/nCoV/POI/%s/intervals_gz/" % country
    save_path = "/home/vsekara/mb_data/cuebiq/nCoV/geostats/%s/" % country

    # check if save path exists
    if os.path.exists(save_path) == False:
        # otherwise create directory
        os.mkdir(save_path)

    # name of files to open
    file = 'all_parts.csv.gz'

    # load urban raster
    urban_raster, urban_band, urban_max_row, urban_max_col = load_urban_raster()
    if country in countries_with_povdata:
        poverty_raster, poverty_band, pov_max_row, pov_max_col = load_poverty_raster(country)
        POVERTY = True
        columns = ['useruuid',admin,'distance_travelled','number_of_pois','radius_of_gyration',
                    'time_at_home','latitude','longitude','time_coverage','urban','poverty']
    else:
        POVERTY = False
        columns = ['useruuid',admin,'distance_travelled','number_of_pois',
                    'radius_of_gyration','time_at_home','latitude','longitude','time_coverage','urban']

    # iterate of days
    while t_day <= t_end:
        day = t_day.strftime('%Y%m%d00')

        # if file does not exist - do analysis
        if os.path.exists(save_path + '%s.csv' % day) == False:

            # print the file it is starting to work on
            print(country,day)

            # load interval data
            interval_data = load_interval_data(interval_path,day,file)
            # load stops data
            stops_data = load_stops_data(stop_path,day,file)
            # create stop_to_admin1 map
            stops_to_admin1 = dict(zip(stops_data['label'],stops_data['GID_2']))
            # create stops to gps pos
            stops_to_pos = dict(zip(stops_data['label'],zip(stops_data['latitude'],stops_data['longitude'])))

            # DISTANCE CALCULATION
            dist = interval_data.groupby('useruuid').agg({'distance':'sum'}).reset_index()

            # HOME LOCATION COUNT
            visits = (interval_data.groupby('useruuid')
                          .agg({'start': list, 'end': list,
                                'classification_type': list,
                                'label': list,
                                'distance':'sum'
                              }).reset_index()[['useruuid','classification_type','label','start','end','distance']].values)

            # after reading file calculate statistics
            save_data = []
            for u,types,labels,starts,ends,dist in visits:

                # unpack list of tuples
                admins = [stops_to_admin1.get(l,-1) for l in labels]
                positions = [stops_to_pos.get(l,-1) for l in labels]

                # calculate data coverage
                time_coverage = sum(np.array(ends) - np.array(starts))

                # find home location
                (home_label,admin_label),time_at_home = infer_home_label(types,labels,admins,starts,ends)

                # calculate radius of gyration
                rg = round(calculate_radius_of_gyration(labels,positions,home_label),3)

                # calculate number of points of interest 
                pois = len([l for l in set(labels) if l!=-1])

                # get urban label of home area
                if home_label!=-1:
                    # find home gps locations
                    lat,lon = map(lambda x: round(x,5), positions[labels.index(home_label)])
                    urban = get_value_from_raster(urban_raster,urban_band,lat,lon,urban_max_row,urban_max_col)
                else:
                    urban = -1
                    lat,lon = float("NaN"),float("NaN")
                save_row = (u,admin_label,round(dist,3),pois,rg,time_at_home,lat,lon,time_coverage,urban)

                # if we have poverty data for the country get poverty of home area
                if POVERTY:
                    if home_label!=-1:
                        poverty = get_value_from_raster(poverty_raster,poverty_band,lat,lon,pov_max_row,pov_max_col)
                    else:
                        poverty = -1
                    save_row += (poverty,)

                # append data to list
                save_data.append(save_row)

            # save file
            pd.DataFrame(save_data,columns=columns).to_csv(save_path + '%s.csv' % day,index=False)

        t_day += timedelta(days=1)

In [ ]:
t_start = datetime(2020,1,1)
#t_end = datetime.now() - timedelta(days = 1)
#t_end = datetime(t_end.year,t_end.month,t_end.day)
t_end = datetime(2020,2,1)

#for country in ['CIV','COL','IDN','IND','MEX','MMR','MOZ','MYS','NGA','UKR']
for country in ['CIV','COL','IDN','IND','MMR','MOZ','MYS','NGA','UKR']:
#for country in ['AUS','DEU','GBR']

#for country in ['MEX']:
    calculate_geostats(country,t_start,t_end,"admin2")

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:98: RuntimeWarning: invalid value encountered in less


CIV 2020010100
CIV 2020010200
CIV 2020010300
CIV 2020010400
CIV 2020010500
CIV 2020010600
CIV 2020010700
CIV 2020010800
CIV 2020010900
CIV 2020011000
CIV 2020011100
CIV 2020011200
CIV 2020011300
CIV 2020011400
CIV 2020011500
CIV 2020011600
CIV 2020011700
CIV 2020011800
CIV 2020011900
CIV 2020012000
CIV 2020012100
CIV 2020012200
CIV 2020012300
CIV 2020012400
CIV 2020012500
CIV 2020012600
CIV 2020012700
CIV 2020012800
CIV 2020012900
CIV 2020013000
CIV 2020013100
COL 2020010100
COL 2020010200
COL 2020010300
COL 2020010400
COL 2020010500
COL 2020010600
COL 2020010700
COL 2020010800
COL 2020010900
COL 2020011000
COL 2020011100
COL 2020011200
COL 2020011300
COL 2020011400
COL 2020011500
COL 2020011600
COL 2020011700
COL 2020011800
COL 2020011900
COL 2020012000
COL 2020012100
COL 2020012200
COL 2020012300
COL 2020012400
COL 2020012500
COL 2020012600
COL 2020012700
COL 2020012800
COL 2020012900
COL 2020013000
COL 2020013100
IDN 2020010100
IDN 2020010200
IDN 2020010300
IDN 2020010400
IDN 202001